# Grid Fluid

Modeling fluid motion by solving the incompressible and inviscid Navier-Stokes equation on a uniform grid

Credit to Bo Zhu, on whose Dartmouth COSC89 course notes the following code has been based.

Additionally, credit to Robert Bridson from the University of British Columbia, whose Fluid Simulation Course Notes were invaluable in understanding the material. His course notes can be found here:\
https://www.cs.ubc.ca/~rbridson/fluidsimulation/fluids_notes.pdf

To have the "source" only add new smoke at the first frame, set `constant_source = False` (found in cell 2). To have the "source" constantly add new smoke, set it to `True`

In [69]:
import taichi as ti
import taichi.math as tm

ti.init()

[Taichi] Starting on arch=x64


In [70]:
### Global Variables ###

substeps = 5
iters = 20
dt = 0.03 
res = 500
dx = 0.015

vel = ti.Vector.field(2, float, shape=(res, res))
vel_copy = ti.Vector.field(2, float, shape=(res,res))
vel_divs = ti.field(float, shape=(res, res))
pressure_hat = ti.field(float, shape=(res, res))
new_pressure = ti.field(float, shape=(res, res))
smoke_density = ti.field(float, shape=(res,res))
den_copy = ti.field(float, shape=(res,res))
vor = ti.field(float, shape=(res,res))
N = ti.Vector.field(2, float, shape=(res, res))

new_src_vel = 3.0
src_pos = tm.vec2(0.5, 0.5)
src_vel = tm.vec2(3, 0)
src_rad = 0.1

colors = ti.Vector.field(3, float, shape=(res,res))

debug = False

# tester variables
smoke_check = ti.field(float, shape=())
vel_check = ti.Vector.field(2, float, shape=())
vel_check_surrounding = ti.Vector.field(2, float, shape=5)
new_pos_check = ti.Vector.field(2, float, shape=())
vor_check = ti.field(float, shape=())
f_N_check = ti.Vector.field(2, float, shape=())
pressure_check = ti.field(float, shape=())
iter_pressure_checks = ti.field(float, shape=iters)

In [71]:
### Helper Functions ###

# @param - coordinates (i,j) of the node
# @return - gradient of p at node (i,j)
@ti.func 
def gradient_pressure(i:ti.i32, j:ti.i32):
    pressures = tm.vec2(pressure_hat[i+1, j] - pressure_hat[i-1, j], pressure_hat[i, j+1] - pressure_hat[i, j-1])
    return pressures / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - divergence of the velocity at grid node (i, j)
@ti.func
def divergence_vel(i:ti.i32, j:ti.i32):
    return (vel[i+1, j][0] - vel[i-1, j][0] + vel[i, j+1][1] - vel[i, j-1][1]) / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - the curl of the velocity at grid node (i, j)
@ti.func
def curl_vel(i:ti.i32, j:ti.i32):
#     if debug and i == 250 and j == 210:
#         vel_check_surrounding[0] = vel[i+1, j]
#         vel_check_surrounding[1] = vel[i-1, j]
#         vel_check_surrounding[2] = vel[i, j+1]
#         vel_check_surrounding[3] = vel[i, j-1]
#     return 1.0
    return (vel[i+1, j][1] - vel[i-1, j][1] - vel[i, j+1][0] + vel[i, j-1][0]) / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - the laplacian of the pressure at grid node (i, j)
@ti.func
def laplacian_pressure(i:ti.i32, j:ti.i32):
    return (pressure_hat[i+1, j] + pressure_hat[i-1, j] + pressure_hat[i, j+1] + pressure_hat[i, j-1] - 4 * pressure_hat[i, j]) / (dx * dx)

# returns whether or not (row, col) is on a boundary
@ti.func
def isBoundary(row:ti.i32, col:ti.i32) -> bool:
    return (row == 0 or col == 0 or row == res-1 or col == res-1)

In [72]:
### Simulator Functions ###

# @param - x_p: vec2, where x_p = (x - u^(n+1/2)(x) * dt) (the "starting point")
# @return - the interpolated value for u*(x) at x_p
@ti.func
def interpolate(x_p, vector_field):
    x = x_p[0]
    y = x_p[1]
    x1 = (int)(ti.floor(x))
    x2 = x1+1
    y1 = (int)(ti.floor(y))
    y2 = y1+1
    # interpolate in x direction
    f_xy1 = (x2 - x)/(x2-x1) * vector_field[x1, y1] + (x - x1)/(x2 - x1) * vector_field[x2, y1]
    f_xy2 = (x2 - x)/(x2-x1) * vector_field[x1, y2] + (x - x1)/(x2 - x1) * vector_field[x2, y2]
    # interpolate in y direction
    f_xy = (y2 - y)/(y2 - y1) * f_xy1 + (y - y1)/(y2 - y1) * f_xy2
    return f_xy

# Fills vel_copy and den_copy
@ti.func
def fill_copies():
    for i in ti.grouped(vel):
        vel_copy[i] = vel[i]
        den_copy[i] = smoke_density[i]

# Advection using a semi-Lagrangian method
@ti.kernel
def advect():
    fill_copies()
    for row in range(res):
        for col in range(res):
            pos = tm.vec2(row, col)
            new_pos = pos - vel[row, col] * 0.5 * dt
            new_vel = interpolate(new_pos, vel_copy)
            new_pos = pos - new_vel * dt
            new_vel = interpolate(new_pos, vel_copy)
            vel[row, col] = new_vel
            
            new_den = interpolate(new_pos, den_copy)
            smoke_density[row, col] = new_den
            
            if debug and row == 250 and col == 200:
                new_pos_check[None] = new_pos
                vel_check[None] = vel[row, col]
                smoke_check[None] = new_den
                vel_check_surrounding[0] = vel[row+1, col]
                vel_check_surrounding[1] = vel[row-1, col]
                vel_check_surrounding[2] = vel[row, col+1]
                vel_check_surrounding[3] = vel[row, col-1]
                vel_check_surrounding[4] = vel[row, col]
    
@ti.func
def swap_pressures():
    for i in ti.grouped(pressure_hat):
        pressure_hat[i] = new_pressure[i]
        
@ti.func
def calc_divs():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                vel_divs[row, col] = divergence_vel(row, col)

@ti.func
def gauss_seidel():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                coef_off_dia = 1.0 / (dx * dx)
                coef_dia = 4.0 / (dx * dx)
                off_diagonal = 0.
                off_diagonal += pressure_hat[row-1, col] + pressure_hat[row+1, col] + pressure_hat[row, col+1] + pressure_hat[row, col-1]
#                 new_pressure[row, col] = (-vel_divs[row, col] + off_diagonal * coef_off_dia) / coef_dia
                pressure_hat[row, col] = (-vel_divs[row, col] + off_diagonal * coef_off_dia) / coef_dia
        
@ti.func
def correct_vel():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                grad_p = gradient_pressure(row, col)
                vel[row, col] -= grad_p
            
                if debug and row == 250 and col == 200:
                    pressure_check[None] = pressure_hat[row, col]
    
@ti.kernel
def project():
    # Calc velocity for RHS of Poissan (div u*)
    calc_divs()
    pressure_hat.fill(0.)
    for i in range(iters):
        gauss_seidel()
        iter_pressure_checks[i] = pressure_hat[250, 250]
#         swap_pressures()
    correct_vel()

@ti.func
def update_vorticity_field():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                vor[row, col] = curl_vel(row, col)

            if debug and row == 250 and col == 210:
#                     vor_check[None] = vor[row, col]
                vor_check[None] = vor[row, col]
#                     i = row
#                     j = col
#                     vel_check_surrounding[0] = vel[i+1, j]
#                     vel_check_surrounding[1] = vel[i-1, j]
#                     vel_check_surrounding[2] = vel[i, j+1]
#                     vel_check_surrounding[3] = vel[i, j-1]


@ti.func
def update_N():
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                N[row, col][0] = (abs(vor[row+1, col]) - abs(vor[row-1, col]))/ (2 * dx)
                N[row, col][1] = (abs(vor[row, col+1]) - abs(vor[row, col-1]))/ (2 * dx)
                N[row, col] /= (N[row, col].norm() + 10e-10)

@ti.func
def cross(v, w):
    return tm.vec2(v[1] * w, -v[0] * w)
                
@ti.func
def update_velocity():
    vor_conf_coef = 8.0
    for row in range(res):
        for col in range(res):
            if not isBoundary(row, col):
                f = vor_conf_coef * dx * cross(N[row, col], vor[row, col])
                if debug and row == 250 and col == 200:
                    f_N_check[None] = f
                vel[row, col] += f * dt
                
@ti.kernel
def vorticity_confinement():
    vor.fill(0.)
    update_vorticity_field()
    update_N() # update N (direction of local rotation)
    update_velocity() # find vorticity confinement force and update velocity

# create a "source" for the smoke
@ti.kernel
def source():
    for row in range(res):
        for col in range(res):
            if (tm.vec2(row/res, col/res) - src_pos).norm() < src_rad:
                vel[row, col] = src_vel
                smoke_density[row, col] = 1

# 0 = up, 1 = left, 2 = down, 3 = right
@ti.kernel
def directed_source(x:ti.f32, y:ti.f32, direction:ti.i32):
    new_source_pos = tm.vec2(x, y)
    for row in range(res):
        for col in range(res):
            if (tm.vec2(row/res, col/res) - new_source_pos).norm() < src_rad:
                if direction == 0: # up
                    vel[row, col] = tm.vec2(0, new_src_vel)
                elif direction == 1: # left
                    vel[row, col] = tm.vec2(-new_src_vel, 0)
                elif direction == 2: # down
                    vel[row, col] = tm.vec2(0, -new_src_vel)
                else: # right
                    vel[row, col] = tm.vec2(new_src_vel, 0)
                smoke_density[row, col] = 1

@ti.kernel
def update_colors():
    for i in ti.grouped(colors):
        curr_den = smoke_density[i]
        colors[i] = tm.vec3(curr_den, curr_den, curr_den)

In [73]:
def substep():
    # renew source for each substep if user asks for it
    if constant_source:
        source()
    advect()
    vorticity_confinement()
    project()
    update_colors()

In [74]:
### Setup ###
@ti.kernel
def init_grid():
    for i in ti.grouped(vel):
        vel[i] = tm.vec2(0,0)
        N[i] = tm.vec2(0,0)
    for i in ti.grouped(smoke_density):
        smoke_density[i] = 0
        vel_divs[i] = 0
        pressure_hat[i] = 0
        new_pressure[i] = 0
        vor[i] = 0

In [75]:
### Function to get print checks ###
@ti.kernel
def get_checks(x:ti.f32, y:ti.f32):
    row = (int)(x * res)
    col = (int)(y * res)
    smoke_check[None] = smoke_density[row, col]
    vel_check[None] = vel[row, col]
    vor_check[None] = vor[row, col]
    pressure_check[None] = pressure_hat[row, col]
    f_N_check[None] = N[row, col]
    vel_check_surrounding[0] = vel[row+1, col]
    vel_check_surrounding[1] = vel[row-1, col]
    vel_check_surrounding[2] = vel[row, col+1]
    vel_check_surrounding[3] = vel[row, col-1]
    vel_check_surrounding[4] = vel[row, col]

def print_checks(x:ti.f32, y:ti.f32):
    row = (int)(x * res)
    col = (int)(y * res)
    print("(row, col) = (%d, %d)" % (row, col))
    print("smoke check", smoke_check[None])
    print("smoke density at middle", smoke_density[250, 250])
    print("vel check", vel_check[None])
    print("vor check", vor_check[None])
    print("N check", f_N_check[None])
    print("pressure check", pressure_check[None])
    print("iter pressure checks", iter_pressure_checks)
    print("surrounding velocities", vel_check_surrounding)


In [76]:
def main():
    gui = ti.GUI('Grid Fluid', (res, res))
    
    visualize_smoke = True  #visualize density (default)
    visualize_vel = False  #visualize velocity
    visualize_div = False # visualize divergence
    
    global constant_source
    constant_source = False
        
    init_grid()
    if not constant_source:
        source()

    if debug:
        for step in range(substeps):
            print("step ", step)
            substep()
            print("smoke check", smoke_check[None])
            print("smoke density at middle", smoke_density[250, 250])
            print("vel check", vel_check[None])
            print("new pos check", new_pos_check[None])
            print("f_N_check", f_N_check[None])
            print("vor check", vor_check[None])
            print("surrounding velocities", vel_check_surrounding)
            print("pressure check", pressure_check[None])
            print("iter pressure checks", iter_pressure_checks)
                    
    while gui.running:
        # Move stuff
        if not debug:
            for step in range(substeps):
                substep()
        
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == ti.GUI.SPACE:
                constant_source = not constant_source
            elif e.key == 'w':
                directed_source(e.pos[0], e.pos[1], 0)
            elif e.key == 'a':
                directed_source(e.pos[0], e.pos[1], 1)
            elif e.key == 's':
                directed_source(e.pos[0], e.pos[1], 2)
            elif e.key == 'd':
                directed_source(e.pos[0], e.pos[1], 3)
            elif e.key == ti.GUI.LMB:
                get_checks(e.pos[0], e.pos[1])
                print_checks(e.pos[0], e.pos[1])
            elif e.key == 'v': # show velocities
                visualize_vel = True
                visualize_smoke = False
                visualize_div = False
            elif e.key == 'b':
                visualize_smoke = True
                visualize_vel = False
                visualize_div = False
            elif e.key == 'g':
                visualize_smoke = False
                visualize_vel = False
                visualize_div = True
            elif e.key == 'c':
                init_grid()
             
        # Draw grid
        if visualize_smoke:
            gui.set_image(colors)
        elif visualize_vel:
            gui.set_image(vel.to_numpy() * 0.01 + 0.5)
        elif visualize_div:
            gui.set_image(vel_divs.to_numpy() * 0.1 + 0.3)
#         gui.set_image(colors)
        gui.show()
        
    
if __name__ == '__main__':
    main()

(row, col) = (259, 244)
smoke check 1.0
smoke density at middle 1.0
vel check [2.7162714  0.01366095]
vor check 7.152558100642636e-05
N check [-0.97618669 -0.21693039]
pressure check 0.0012331551406532526
iter pressure checks [-2.67626747e-05 -5.34932733e-05 -8.01915085e-05 -1.06857406e-04
 -1.33491121e-04 -1.60092619e-04 -1.86661870e-04 -2.13198844e-04
 -2.39703528e-04 -2.66175863e-04 -2.92615878e-04 -3.19023529e-04
 -3.45398817e-04 -3.71741800e-04 -3.98052274e-04 -4.24330356e-04
 -4.50575986e-04 -4.76789282e-04 -5.02970070e-04 -5.29118406e-04]
surrounding velocities [[2.7091243  0.01550913]
 [2.7224278  0.01181065]
 [2.7179594  0.01142273]
 [2.7142632  0.01588394]
 [2.7162714  0.01366095]]
(row, col) = (406, 155)
smoke check 0.0
smoke density at middle 1.0
vel check [0.4848648  1.37700748]
vor check -61.52188491821289
N check [ 0.2183155  -0.97587818]
pressure check 0.0014397513587027788
iter pressure checks [-1.1721390e-24 -2.2560253e-23 -2.2592709e-22 -1.5676620e-21
 -8.4691356e-21

(row, col) = (305, 261)
smoke check 0.9999996423721313
smoke density at middle 1.0
vel check [2.82830286 0.17262666]
vor check -0.6738584041595459
N check [-0.47265276  0.88124871]
pressure check 0.004264810122549534
iter pressure checks [-1.0280150e-24 -1.9414555e-23 -1.9088212e-22 -1.3011688e-21
 -6.9102281e-21 -3.0464984e-20 -1.1602221e-19 -3.9221193e-19
 -1.2002929e-18 -3.3751285e-18 -8.8217001e-18 -2.1631221e-17
 -5.0134742e-17 -1.1051572e-16 -2.3291611e-16 -4.7139485e-16
 -9.1965395e-16 -1.7351598e-15 -3.1751752e-15 -5.6492939e-15]
surrounding velocities [[ 2.8778522   0.20694478]
 [ 2.8957527   0.20479438]
 [ 2.995885   -0.448745  ]
 [ 2.949921   -0.18884625]
 [ 2.8283029   0.17262666]]
(row, col) = (340, 291)
smoke check 0.9064511656761169
smoke density at middle 1.0
vel check [-0.34377742  0.73553765]
vor check -131.04544067382812
N check [0.58639109 0.81002802]
pressure check 0.003364198375493288
iter pressure checks [-4.6766652e-25 -8.8998877e-24 -8.8203712e-23 -6.0625589e-2

## Todo

make it so you can display divergence